<a href="https://colab.research.google.com/github/dfukagaw28/ColabNotebooks/blob/main/%E6%96%87%E5%AD%97%E3%82%B3%E3%83%BC%E3%83%89%E3%81%AB%E3%81%A4%E3%81%84%E3%81%A6_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 文字コードについて (2)

*   Python の encode/decode でいわゆる「シフトJIS」を扱う場合，新しい規格である「shift_jis_2004」よりも，広く利用されている「cp932」を使う方が望ましい場合が多い。
*   近年では「シフトJIS」でなく「UTF-8」を利用することが推奨されており，今後も（少なく見積もっても10～20年くらいは）その傾向が続くはず。
*   つまり，「shift_jis_2004」の出番はあまりないかもしれない。

## python の encode/decode で文字コードを変換する

*   [Python で扱える文字コードの一覧](https://docs.python.org/ja/3/library/codecs.html#standard-encodings)

In [1]:
import itertools
import pandas as pd

In [2]:
def enum_characters(encoding, length=2, check_reverse=False):
  queue = [bytes([x]) for x in range(0x100)]
  while queue:
    text_encoded = queue.pop(0)
    try:
      text = text_encoded.decode(encoding)
      if check_reverse:
        text_encoded2 = text.encode(encoding)
        assert text_encoded == text_encoded2
      yield text_encoded
    except:
      if len(text_encoded) < length:
        queue += [text_encoded + bytes([x]) for x in range(0x100)]

In [3]:
encodings = ['shift_jis', 'cp932', 'shift_jisx0213', 'shift_jis_2004']
df = pd.DataFrame(index=encodings)
df['1byte'] = [len([*enum_characters(encoding, 1)]) for encoding in encodings]
df['2bytes'] = [len([*enum_characters(encoding, 2)]) for encoding in encodings]
df['1byte (rev)'] = [len([*enum_characters(encoding, 1, check_reverse=True)]) for encoding in encodings]
df['2bytes (rev)'] = [len([*enum_characters(encoding, 2, check_reverse=True)]) for encoding in encodings]
df['characters'] = [len(set(x.decode(encoding) for x in enum_characters(encoding, 2))) for encoding in encodings]
df

,1byte,2bytes,1byte (rev),2bytes (rev),characters
shift_jis,191,7070,191,7070,7070
cp932,196,9800,196,9402,9402
shift_jisx0213,191,11414,191,11414,11414
shift_jis_2004,191,11424,191,11424,11424


## cp932 は1対1対応でない

In [4]:
encoding = 'cp932'
cp932_all = [*enum_characters(encoding, 2)]
cp932_rev = [*enum_characters(encoding, 2, check_reverse=True)]

chars = sorted(set(cp932_all) - set(cp932_rev))
chars_hex = [char.hex() for char in chars]
df = pd.DataFrame(index=chars_hex)
for char in chars:
  df.loc[char.hex(), 'char1'] = char.decode('cp932')
  df.loc[char.hex(), 'rev'] = char.decode('cp932').encode('cp932').hex()
  df.loc[char.hex(), 'char2'] = char.decode('cp932').encode('cp932').decode('cp932')
df

,char1,rev,char2
8790,≒,81e0,≒
8791,≡,81df,≡
8792,∫,81e7,∫
8795,√,81e3,√
8796,⊥,81db,⊥
...,...,...,...
fc47,鵰,eee8,鵰
fc48,鵫,eee9,鵫
fc49,鶴,eeea,鶴
fc4a,鸙,eeeb,鸙


## shift_jis_2004 と shift_jisx0213 との差分

*   shift_jis_2004 になって 10 文字増えた。

In [5]:
sjis2004_bytes = [*enum_characters('shift_jis_2004', 2)]
sjisx0213_bytes = [*enum_characters('shift_jisx0213', 2)]

print(set(sjis2004_bytes) - set(sjisx0213_bytes))
print(set(sjisx0213_bytes) - set(sjis2004_bytes))

for x in sorted(set(sjis2004_bytes) - set(sjisx0213_bytes)):
  print(x.hex(), x.decode('shift_jis_2004'))

{b'\x98\x9e', b'\x88\x9e', b'\xef\xfb', b'\xef\xf8', b'\xea\xa5', b'\xef\xfa', b'\xef\xf9', b'\x87\x9f', b'\x98s', b'\xef\xfc'}
set()
879f 俱
889e 剝
9873 𠮟
989e 吞
eaa5 噓
eff8 姸
eff9 屛
effa 幷
effb 瘦
effc 繫


## shift_jisx0213 と cp932 との差分

*   表現可能な文字の違いを確認する

In [6]:
sjisx0213_bytes = [*enum_characters('shift_jisx0213', 2)]
cp932_bytes = [*enum_characters('cp932', 2)]

print(len(set(sjisx0213_bytes) - set(cp932_bytes)))
print(len(set(cp932_bytes) - set(sjisx0213_bytes)))
print()

for x in sorted(set(sjisx0213_bytes) - set(cp932_bytes))[:10]:
  print(x.hex(), x.decode('shift_jisx0213'))
print()

for x in sorted(set(cp932_bytes) - set(sjisx0213_bytes))[:10]:
  print(x.hex(), x.decode('cp932'))

1629
15

81ad ＇
81ae ＂
81af －
81b0 ~
81b1 〳
81b2 〴
81b3 〵
81b4 〻
81b5 〼
81b6 ヿ

80 
8790 ≒
8791 ≡
8792 ∫
8794 ∑
8795 √
8796 ⊥
8797 ∠
879a ∵
879b ∩


## PythonのシフトJIS関連の文字集合

In [7]:
encodings = ['shift_jis', 'cp932', 'shift_jisx0213', 'shift_jis_2004']

sjis_sets = {encoding: set([x.decode(encoding) for x in enum_characters(encoding, 2)]) for encoding in encodings}

all_characters = list(set.union(*sjis_sets.values()))
all_characters.sort()

df = pd.DataFrame(index=all_characters, columns=['shift_jis', 'cp932', 'shift_jisx0213', 'shift_jis_2004'])

for char in all_characters:
  for encoding in encodings:
    df.loc[char, encoding] = char in sjis_sets[encoding]

df['count'] = 1

df_agg = df.groupby(encodings).agg('sum')

df_agg

count
shift_jis cp932 shift_jisx0213 shift_jis_2004       
False     False False          True               11
                True           False               1
                               True             3981
          True  False          False            1975
                True           True              363
True      False True           True                6
          True  False          False               1
                True           True             7063

*   基本的に shift_jis_2004 は shift_jisx0213 の上位互換のはず
    *   しかも差分は 10 文字（いずれも JIS 第 3 水準）のはず
    *   Python の encode/decode では，追加 10 文字に加えて，1 文字だけ変更がある
        *   U+9B1C（鬜） と U+9B1D（鬝）
        *   https://www.asahi-net.or.jp/~ax2s-kmtn/ref/jisx0213/ 「JIS X 0213:2004 追補1 p. 31」による変更らしい
*   もっとも文字集合として広いのは shift_jis_2004 だが，shift_jis_2004 には無く cp932 には存在する文字がいくつかある
    *   代表的なものは「髙（はしごだか）」
*   さらに，よく利用される文字についても cp932 と shift_jis_2004 で違いがみられる
    *   たとえば「U+FA11 﨑（たちさき）」は cp932 では `ED 95` だが，これを shift_jis_2004 でデコードすると「U+7CBC 粼」になってしまう。
*   つまり，青空文庫などで用いられている「シフト JIS」を正しく読み込むには shift_jis_2004 よりも cp932 の方が適していると考えられる。


### shift_jis_2004 にしかない文字（11文字）

In [8]:
df2 = df.reset_index().set_index(encodings).sort_index()
df2 = df2[['index']].rename(columns={'index':'char'})
df2['utf8'] = df2['char'].apply(lambda c: c.encode('utf8').hex())

In [9]:
df2.loc[(False, False, False, True)]

char      utf8
shift_jis cp932 shift_jisx0213 shift_jis_2004               
False     False False          True              俱    e4bfb1
                               True              剝    e5899d
                               True              吞    e5909e
                               True              噓    e59993
                               True              姸    e5a7b8
                               True              屛    e5b19b
                               True              幷    e5b9b7
                               True              瘦    e798a6
                               True              繫    e7b9ab
                               True              鬜    e9ac9c
                               True              𠮟  f0a0ae9f

### shift_jisx0213 にしかない文字（1文字）

In [10]:
df2.loc[(False, False, True, False)]

,,,,char,utf8
shift_jis,cp932,shift_jisx0213,shift_jis_2004,,
False,False,True,False,鬝,e9ac9d


### cp932 にしかない文字（1975文字）

*   はしごだか「髙」など

In [11]:
sorted(list(df2.loc[(False, True, False, False)]['char']))

['\x80',
 '∑',
 '仼',
 '伹',
 '俍',
 '俿',
 '僘',
 '僴',
 '兤',
 '冾',
 '凬',
 '劜',
 '勀',
 '卲',
 '叝',
 '坙',
 '坥',
 '墲',
 '奓',
 '奣',
 '妺',
 '峵',
 '巐',
 '弡',
 '恝',
 '悅',
 '惞',
 '愑',
 '愠',
 '戓',
 '敎',
 '昮',
 '昻',
 '朎',
 '櫢',
 '汯',
 '浯',
 '涖',
 '淲',
 '淸',
 '渹',
 '猤',
 '玽',
 '珒',
 '珵',
 '琩',
 '皂',
 '硺',
 '羡',
 '菶',
 '蕫',
 '蠇',
 '譓',
 '赶',
 '軏',
 '遧',
 '釞',
 '鈆',
 '鉷',
 '鋕',
 '鎤',
 '鏸',
 '鐱',
 '鑈',
 '閒',
 '靃',
 '靑',
 '餧',
 '髙',
 '鮻',
 '\ue000',
 '\ue001',
 '\ue002',
 '\ue003',
 '\ue004',
 '\ue005',
 '\ue006',
 '\ue007',
 '\ue008',
 '\ue009',
 '\ue00a',
 '\ue00b',
 '\ue00c',
 '\ue00d',
 '\ue00e',
 '\ue00f',
 '\ue010',
 '\ue011',
 '\ue012',
 '\ue013',
 '\ue014',
 '\ue015',
 '\ue016',
 '\ue017',
 '\ue018',
 '\ue019',
 '\ue01a',
 '\ue01b',
 '\ue01c',
 '\ue01d',
 '\ue01e',
 '\ue01f',
 '\ue020',
 '\ue021',
 '\ue022',
 '\ue023',
 '\ue024',
 '\ue025',
 '\ue026',
 '\ue027',
 '\ue028',
 '\ue029',
 '\ue02a',
 '\ue02b',
 '\ue02c',
 '\ue02d',
 '\ue02e',
 '\ue02f',
 '\ue030',
 '\ue031',
 '\ue032',
 '\ue033',
 '\ue

### shift_jis や cp932 には無いが shift_jisx0213 にも shift_jis_2004 には含まれる（3981文字）

*   丸付き数字の 21～50 など
*   アクセント付き英字や多くの記号が含まれる

In [12]:
sorted(list(df2.loc[(False, False, True, True)]['char']))

['\xa0',
 '¡',
 '¤',
 '¥',
 '¦',
 '©',
 'ª',
 '«',
 '\xad',
 '®',
 '¯',
 '²',
 '³',
 '·',
 '¸',
 '¹',
 'º',
 '»',
 '¼',
 '½',
 '¾',
 '¿',
 'À',
 'Á',
 'Â',
 'Ã',
 'Ä',
 'Å',
 'Æ',
 'Ç',
 'È',
 'É',
 'Ê',
 'Ë',
 'Ì',
 'Í',
 'Î',
 'Ï',
 'Ð',
 'Ñ',
 'Ò',
 'Ó',
 'Ô',
 'Õ',
 'Ö',
 'Ø',
 'Ù',
 'Ú',
 'Û',
 'Ü',
 'Ý',
 'Þ',
 'ß',
 'à',
 'á',
 'â',
 'ã',
 'ä',
 'å',
 'æ',
 'æ̀',
 'ç',
 'è',
 'é',
 'ê',
 'ë',
 'ì',
 'í',
 'î',
 'ï',
 'ð',
 'ñ',
 'ò',
 'ó',
 'ô',
 'õ',
 'ö',
 'ø',
 'ù',
 'ú',
 'û',
 'ü',
 'ý',
 'þ',
 'ÿ',
 'Ā',
 'ā',
 'Ă',
 'ă',
 'Ą',
 'ą',
 'Ć',
 'ć',
 'Ĉ',
 'ĉ',
 'Č',
 'č',
 'Ď',
 'ď',
 'đ',
 'Ē',
 'ē',
 'Ę',
 'ę',
 'Ě',
 'ě',
 'Ĝ',
 'ĝ',
 'Ĥ',
 'ĥ',
 'ħ',
 'Ī',
 'ī',
 'Ĵ',
 'ĵ',
 'Ĺ',
 'ĺ',
 'Ľ',
 'ľ',
 'Ł',
 'ł',
 'Ń',
 'ń',
 'Ň',
 'ň',
 'ŋ',
 'Ō',
 'ō',
 'Ő',
 'ő',
 'Œ',
 'œ',
 'Ŕ',
 'ŕ',
 'Ř',
 'ř',
 'Ś',
 'ś',
 'Ŝ',
 'ŝ',
 'Ş',
 'ş',
 'Š',
 'š',
 'Ţ',
 'ţ',
 'Ť',
 'ť',
 'Ū',
 'ū',
 'Ŭ',
 'ŭ',
 'Ů',
 'ů',
 'Ű',
 'ű',
 'Ź',
 'ź',
 'Ż',
 'ż',
 'Ž',
 'ž',
 'Ɠ',
 'ǂ',
 'Ǎ',
 'ǎ

## たちさき「﨑」について

In [13]:
'﨑'.encode('cp932'), '﨑'.encode('shift_jisx0213'), '﨑'.encode('shift_jis_2004')

(b'\xed\x95', b'\x98\x92', b'\x98\x92')

In [14]:
[b'\xed\x95'.decode(encoding) for encoding in ['cp932', 'shift_jisx0213', 'shift_jis_2004']]

['﨑', '粼', '粼']

In [15]:
[b'\x98\x92'.decode(encoding) for encoding in ['shift_jisx0213', 'shift_jis_2004']]

['﨑', '﨑']